In [26]:
import json

# Constants
ADMINS_FILE = 'admins.json'
MEMBERS_FILE = 'members.json'
PRODUCTS_FILE = 'products.json'
ORDERS_FILE = 'orders.json'

# Load data from files
def load_data(filename):
    try:
        with open(filename, 'r') as file:
            data = json.load(file)
        return data
    except FileNotFoundError:
        return []

# Save data to files
def save_data(data, filename):
    with open(filename, 'w') as file:
        json.dump(data, file, indent=4)

# Registration Functionality
def register_user(user_type, full_name, address, email, password):
    if user_type == 'admin':
        admins = load_data(ADMINS_FILE)
        admins.append({"Full Name": full_name, "Address": address, "Email": email, "Password": password})
        save_data(admins, ADMINS_FILE)
    elif user_type == 'member':
        members = load_data(MEMBERS_FILE)
        members.append({"Full Name": full_name, "Address": address, "Email": email, "Password": password})
        save_data(members, MEMBERS_FILE)

# Login Functionality
def login_user(email, password):
    admins = load_data(ADMINS_FILE)
    members = load_data(MEMBERS_FILE)
    for admin in admins:
        if admin['Email'] == email and admin['Password'] == password:
            return 'admin', admin
    for member in members:
        if member['Email'] == email and member['Password'] == password:
            return 'member', member
    return None, None

# Create/Update Product Functionality
def create_or_update_product(admin, product_id, product_name, manufacturer_name, price, discount, total_stock):
    products = load_data(PRODUCTS_FILE)
    for product in products:
        if product['Product ID'] == product_id:
            product.update({"Product Name": product_name, "Manufacturer Name": manufacturer_name, "Price": price,
                            "Discount": discount, "Total Stock Available": total_stock})
            break
    else:
        products.append({"Created By": admin['Full Name'], "Product ID": product_id, "Product Name": product_name,
                         "Manufacturer Name": manufacturer_name, "Price": price, "Discount": discount,
                         "Total Stock Available": total_stock})
    save_data(products, PRODUCTS_FILE)

# Product Listing Functionality
def get_products_list(admin):
    products = load_data(PRODUCTS_FILE)
    admin_products = [product for product in products if product['Created By'] == admin['Full Name']]
    return admin_products

# Create Order Functionality
def create_order(member, product_id, quantity):
    products = load_data(PRODUCTS_FILE)
    for product in products:
        if product['Product ID'] == product_id and product['Total Stock Available'] >= quantity:
            order_id = f"ORD{len(load_data(ORDERS_FILE)) + 1}"
            total_cost = (product['Price'] * quantity) * (1 - float(product['Discount'][:-1]) / 100)
            order_data = {
                "Order ID": order_id,
                "Product Name": product['Product Name'],
                "Price": product['Price'],
                "Discount": product['Discount'],
                "Price after Discount": total_cost,
                "Quantity": quantity,
                "Total Cost": total_cost * quantity,
                "Ordered By": member['Full Name'],
                "Delivering to": member['Address']
            }
            orders = load_data(ORDERS_FILE)
            orders.append(order_data)
            save_data(orders, ORDERS_FILE)
            product['Total Stock Available'] -= quantity
            save_data(products, PRODUCTS_FILE)
            return order_id
    return None

# Sample Usage
# Registering a member
register_user('member', 'Viru Sahastrabudhhe', 'Villa 25, Imperial College, New Delhi India', 'Virus@gmail.com', 'SilencerIsTheBest')

# Logging in a member
user_type, user_data = login_user('Virus@gmail.com', 'SilencerIsTheBest')
if user_type == 'member':
    print(f"Welcome, {user_data['Full Name']}!")
    # Creating an order
    order_id = create_order(user_data, 'GTRX1', 2)
    if order_id:
        print(f"Order placed successfully! Order ID: {order_id}")
    else:
        print("Failed to place the order. Please check product availability.")
else:
    print("Invalid credentials.")


Welcome, Viru Sahastrabudhhe!
Order placed successfully! Order ID: ORD7
